In [ ]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import NMF;
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn import svm
from sklearn import tree
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/M.Tech/Projects/Effective\ Feature\ Extraction/ Dataset/

In [ ]:
col_names = np.array(["protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","labels","port"])

data = pd.read_csv('NSL-KDD/KDDTrain+.txt', names=col_names)

In [ ]:
data['labels'].value_counts()

normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: labels, dtype: int64

In [ ]:
#data.isna().sum()

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
  
data['protocol_type']= label_encoder.fit_transform(data['protocol_type']) 
data['service']= label_encoder.fit_transform(data['service']) 
data['flag']= label_encoder.fit_transform(data['flag']) 
data['labels']= label_encoder.fit_transform(data['labels']) 

In [ ]:
data.head()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,port
0,1,20,9,491,0,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,11,20
0,2,44,9,146,0,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,11,15
0,1,49,5,0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,9,19
0,1,24,9,232,8153,0,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,11,21
0,1,24,9,199,420,0,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11,21


In [ ]:
data.describe()

,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels,port
count,125973.000000,125973.000000,125973.000000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,1.053202,31.226469,6.979996,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,0.001222,...,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240,10.625483,19.504060
std,0.426620,16.346470,2.689365,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,0.045239,...,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459,2.636084,2.291503
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,5.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,18.000000
50%,1.000000,24.000000,9.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,20.000000
75%,1.000000,49.000000,9.000000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000,11.000000,21.000000
max,2.000000,69.000000,10.000000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,22.000000,21.000000


In [ ]:
data = data.drop(columns=['land'], axis=1).copy()
data = data.drop(columns=['urgent'], axis=1).copy()
data.shape

(125973, 40)

In [ ]:
y=data['labels']
X = data.drop(['labels'], axis = 1)
X = pd.get_dummies(X, prefix_sep='_')

In [ ]:
transformer = TfidfTransformer(smooth_idf=False)
x_tfidf = transformer.fit_transform(X)
#tfidf.toarray()

/root/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1466: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


In [ ]:
X.shape

(125973, 39)

In [ ]:
#obtain a NMF model.
model = NMF(n_components=30, init='random');
NMfc = model.fit(X,y).transform(X)

In [ ]:
model = NMF(n_components=36, init='nndsvd');
NMfc = model.fit(X,y).transform(X)

model1 = NMF(n_components=33, init='nndsvd');
NMfc1 = model1.fit(NMfc,y).transform(NMfc)

model2 = NMF(n_components=30, init='nndsvd');
NMfc2 = model2.fit(NMfc1,y).transform(NMfc1)

model3 = NMF(n_components=27, init='nndsvd');
NMfc3 = model3.fit(NMfc2,y).transform(NMfc2)

In [ ]:
print(NMfc3.shape)

(125973, 27)


In [ ]:
#Univariate features Selection 

#apply SelectKBest class to extract top 25 best features
bestfeatures = SelectKBest(score_func=chi2, k=20)
fit = bestfeatures.fit(NMfc3,y)
features = fit.transform(NMfc3)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']  #naming the dataframe columns
X_uni=featureScores.nlargest(25,'Score')
print(featureScores.nlargest(25,'Score'))  #print 25 best features

              Features        Score
3            src_bytes  1741.455979
8            logged_in   699.093225
5       wrong_fragment   512.514264
0        protocol_type   271.988770
1              service   235.546002
2                 flag   221.887395
9      num_compromised   168.066736
12            num_root   145.613339
13  num_file_creations   139.581554
22     srv_serror_rate   132.796942
18      is_guest_login   122.886396
15    num_access_files   118.005650
7    num_failed_logins   116.568178
16   num_outbound_cmds   103.654526
23         rerror_rate    91.297888
6                  hot    70.416091
19               count    52.252607
21         serror_rate    47.983475
25       same_srv_rate    44.990403
10          root_shell    43.343551
17       is_host_login    28.065740
11        su_attempted    17.932926
14          num_shells    15.754193
20           srv_count    12.415311
26       diff_srv_rate     9.208504


In [ ]:
features.shape

(125973, 20)

In [ ]:
# normalize the data attributes
normalized_X = preprocessing.normalize(features)
normalized_Xnmf = preprocessing.normalize(NMfc)

In [ ]:
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(normalized_X,y,test_size=0.20, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
#Gaussian Naive Baye Model
BNB_Classifier = BernoulliNB()
BNB_Classifier.fit(X_train, y_train)

BNB_pred=BNB_Classifier.predict(X_test)
accuracy = accuracy_score(BNB_pred,y_test)
print(BNB_Classifier)
print(classification_report(BNB_pred, y_test, labels=None))
print("BernoulliNB_accuracy = ",accuracy*100)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
              precision    recall  f1-score   support

           0       0.31      0.56      0.40       101
           1       0.33      0.43      0.38         7
           2       0.00      0.00      0.00         6
           3       0.27      0.04      0.07        77
           4       1.00      0.20      0.33         5
           5       0.74      0.60      0.66       908
           6       0.00      0.00      0.00         2
           9       0.94      0.92      0.93      8483
          10       0.43      0.45      0.44       302
          11       0.93      0.95      0.94     13207
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.02      0.09      0.04        11
          15       0.80      0.77      0.78       600
          16       1.00      0.07      0.13        14
          17       0.53      0.54      0.53       729
          

/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# LogisticRegression Model
LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=0)
LGR_Classifier.fit(X_train, y_train);

LGR_pred=LGR_Classifier.predict(X_test)
accuracy = accuracy_score(LGR_pred,y_test)
print(LGR_Classifier)
print(classification_report(LGR_pred, y_test, labels=None))
print("LogisticRegression_accuracy = ",accuracy*100)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=0,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
              precision    recall  f1-score   support

           0       0.91      0.92      0.92       182
           1       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.74      0.66      0.70       818
           6       0.00      0.00      0.00         0
           9       0.98      0.97      0.97      8321
          10       0.01      0.33      0.02         9
          11       0.99      0.93      0.96     14228
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00      

/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#SVM model
from sklearn import svm
svm = svm.SVC()
svm = svm.fit(X_train,y_train)

pred_svm = svm.predict(X_test)
accuracy = accuracy_score(pred_svm, y_test)
print(svm)
print(classification_report(pred_svm, y_test, labels=None))
print("SVM_accuracy = ",accuracy*100)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       189
           1       0.22      1.00      0.36         2
           3       0.45      0.83      0.59         6
           4       1.00      1.00      1.00         1
           5       0.85      0.71      0.77       884
           6       0.00      0.00      0.00         0
           9       0.99      0.98      0.99      8339
          10       0.31      0.88      0.46       111
          11       0.99      0.96      0.98     13786
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.80      0.88      0.84    

/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Train KNeighborsClassifier Model
KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
KNN_Classifier.fit(X_train, y_train); 

KNN_pred=KNN_Classifier.predict(X_test)
accuracy = accuracy_score(KNN_pred,y_test)
print(KNN_Classifier)
print(classification_report(KNN_pred, y_test, labels=None))
print("KNN_accuracy = ",accuracy*100)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       185
           1       0.22      1.00      0.36         2
           3       0.73      0.73      0.73        11
           4       1.00      1.00      1.00         1
           5       0.93      0.80      0.86       849
           6       0.33      0.20      0.25         5
           9       0.99      0.99      0.99      8231
          10       0.60      0.79      0.68       239
          11       0.99      0.99      0.99     13427
          12       0.00      0.00      0.00         0
          13       1.00      1.00      1.00         1
          14       0.67      0.91      0.77        32
          15       0.91      0.94      0.93       553
          16       0.00      0.00      0.00         2
    

/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#DecisionTreeClassifier
clf = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
# accuracy
accuracy = accuracy_score(pred, y_test)
print(clf)
print(classification_report(pred, y_test, labels=None))
print("DecisionTree_accuracy = ",accuracy*100)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       184
           1       0.22      0.67      0.33         3
           3       0.73      1.00      0.84         8
           4       1.00      0.25      0.40         4
           5       0.88      0.89      0.89       723
           6       0.33      0.25      0.29         4
           7       0.00      0.00      0.00         3
           9       1.00      0.99      1.00      8237
          10       0.75      0.74      0.75       320
          11       1.00      1.0

/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Random forest
R_clf = RandomForestClassifier()
R_clf.fit(X_train,y_train)

R_pred = R_clf.predict(X_test)
accuracy = accuracy_score(R_pred,y_test)
print(R_clf)
print(classification_report(R_pred, y_test, labels=None))
print("RandomForest_accuracy = ",accuracy*100)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       185
           1       0.33      1.00      0.50         3
           3       0.73      1.00      0.84         8
           4       1.00      1.00      1.00         1
           5       0.95      0.88      0.91       785
           6       0.33      1.00      0.50         1
           7       0.00      0.00      0.00         2
           9       1

/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
